# Предсказание рейтинга ресторана на сайте TripAdvisor
Есть сведения о 50 000 ресторанах Европы, взятые с сайта TripAdvisor

**Цель**: самостоятельно очистить довольно сильно загрязнённый датасет, извлечь из него несколько новых признаков и подготовить данные для обучения модель.  Есть уже готовая модель, которая будет предсказывать рейтинг ресторана по данным сайта TripAdvisor на основе имеющихся в датасете данных.

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import os
from sklearn.model_selection import train_test_split
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
df_train = pd.read_csv(
    '../input/sf-dst-restaurant-rating/main_task.csv')
df_test = pd.read_csv(
    '../input/sf-dst-restaurant-rating/kaggle_task.csv')
sample_submission = pd.read_csv(
    '../input/sf-dst-restaurant-rating/sample_submission.csv')

In [ ]:
# нашел датасет со столицами стран на kaggle, буду использовать для создания нового признака - является ли город столицей
capital = pd.read_csv('../input/coccap/concap.csv')

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1  # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
# в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями
df_test['Rating'] = 0

data = df_test.append(df_train, sort=False).reset_index(
    drop=True)  # объединяем

In [ ]:
# посмотрим на типы данных и количество пропусков
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
fig, ax = plt.subplots(figsize=(20, 12))
sns_heatmap = sns.heatmap(
    data.isnull(), yticklabels=False, cbar=False, cmap='viridis')

**Целевая переменная (Rating)** - без пропусков

**Признаки без пропусков:**
* Restaurant_id
* City
* Ranking
* Reviews
* URL_TA
* ID_TA

**Числовой формат данных:**
* Ranking
* Rating - *целевая переменная*
* Number of Reviews

**Данные в смешанном формате** (похоже на список,но по факту - строка, есть дата и т.д.)
* Cuisine Style
* Reviews

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data

## Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# В датасете 3 признака с пропусками, по всем 3м вынесем отсутствие информации в отдельный признак
data['Number_of_Reviews_isNAN'] = pd.isna(
    data['Number of Reviews']).astype('uint8')
data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')
data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')

**Number of Reviews** - пробовала заполнять пропуски модой, медианой, средним и нулем, лучшее значение МАЕ при заполнении нулем. Предположительно, пропуски - это случаи, когда клиент не написал отзыв, только поставил оценку

In [ ]:
data['Number of Reviews'].fillna(0, inplace=True)

**Price Range** посмотрим на этот признак, какие он принимает значения и сколько пропусков

In [ ]:
data['Price Range'].value_counts(dropna = False)

Признак ординальный (последовательный), нулем/средним заполнить не получится, заполним самым часто встречающимся значением, то есть средним ценовым диапазоном

In [ ]:
data['Price Range'] = data['Price Range'].fillna(data['Price Range'].mode()[0])

**Cuisine Style** сложный признак, тип данных строка, а по факту это список. Заполним пропуски значением Other, позднее разберем на составляющие

In [ ]:
data['Cuisine Style'] = data['Cuisine Style'].fillna("['Other']")

## Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

**интересное наблюдение** - Restaurant_id не уникальный признак и видимо не несет никакой ценной информации, удалим его

In [ ]:
data.drop(['Restaurant_id'], axis=1, inplace=True)

### создадим новый признак Столица (бинарный) 

In [ ]:
# сделаем из колонки в найденном на kaggle датафрейме (сериз) - список, так удобнее искать
caplist = capital.CapitalName.to_list()
caplist

In [ ]:
data['is_capital'] = data.City.apply(lambda x: 1 if x in caplist else 0)

### добавим новый признак - кол-во ресторанов в городе

посмотрим, сколько ресторанов в каждом из 31 городе

In [ ]:
data['City'].value_counts()

In [ ]:
rest_in_city = data['City'].value_counts()
data['Rest_in_city'] = data['City'].apply(lambda x: rest_in_city[x])

### добавим новый признак - Население в городе
городов всего 31 штука, создаем датасет с числом жителей в городах  

In [ ]:
population = {
    'London': 8173900,
    'Paris': 2240621,
    'Madrid': 3155360,
    'Barcelona': 1593075,
    'Berlin': 3326002,
    'Milan': 1331586,
    'Rome': 2870493,
    'Prague': 1272690,
    'Lisbon': 547733,
    'Vienna': 1765649,
    'Amsterdam': 825080,
    'Brussels': 144784,
    'Hamburg': 1718187,
    'Munich': 1364920,
    'Lyon': 496343,
    'Stockholm': 1981263,
    'Budapest': 1744665,
    'Warsaw': 1720398,
    'Dublin': 506211,
    'Copenhagen': 1246611,
    'Athens': 3168846,
    'Edinburgh': 476100,
    'Zurich': 402275,
    'Oporto': 221800,
    'Geneva': 196150,
    'Krakow': 756183,
    'Oslo': 673469,
    'Helsinki': 574579,
    'Bratislava': 413192,
    'Luxembourg': 576249,
    'Ljubljana': 277554
}

In [ ]:
data['Population'] = data['City'].map(population)

### добавим новый признак - рестораннообеспеченность или кол-во ресторанов на человека

In [ ]:
data['Rest_per_man'] = data['Population'] / data['Rest_in_city']

### добавим новый признак - Country
городов немного, создаем словарь 

In [ ]:
country = {
    'London': 'GB',
    'Paris': 'FR',
    'Madrid': 'ES',
    'Barcelona': 'ES',
    'Berlin': 'DE',
    'Milan': 'IT',
    'Rome': 'IT',
    'Prague': 'CZ',
    'Lisbon': 'PT',
    'Vienna': 'AT',
    'Amsterdam': 'NL',
    'Brussels': 'BE',
    'Hamburg': 'DE',
    'Munich': 'DE',
    'Lyon': 'FR',
    'Stockholm': 'SE',
    'Budapest': 'HU',
    'Warsaw': 'PL',
    'Dublin': 'IE',
    'Copenhagen': 'DK',
    'Athens': 'GR',
    'Edinburgh': 'GB',
    'Zurich': 'CH',
    'Oporto': 'PT',
    'Geneva': 'CH',
    'Krakow': 'PL',
    'Oslo': 'NO',
    'Helsinki': 'FI',
    'Bratislava': 'SK',
    'Luxembourg': 'LU',
    'Ljubljana': 'SI'
}

data['Country'] = data['City'].apply(lambda x: country[x])

посмотрим, сколько получилось стран

In [ ]:
data['Country'].nunique()

и сколько ресторанов в каждой стране

In [ ]:
data['Country'].value_counts()

### добавим новый признак - кол-во ресторанов в стране

In [ ]:
rest_in_country = data['Country'].value_counts()
data['Rest_in_country'] = data['Country'].apply(lambda x: rest_in_country[x])

### добавим новый признак, точней 31 новый признак по городам

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
# Add a column to indicate NaNs, if False NaNs are ignored.
data = pd.get_dummies(data, columns=['City', ], dummy_na=True)

### добавим новый признак, точней 22 новых признака по странам

In [ ]:
data = pd.get_dummies(data, columns=[ 'Country',], dummy_na=True)

__Возьмем следующий признак "Price Range"__

In [ ]:
data['Price Range'].value_counts()

### создадим новый признак, точней 3 признака для каждого диапазона цен

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
data['Price Range_Cat'] = labelencoder.fit_transform(data['Price Range'])

__рассмотрим следующий признак Reviews__
внутри этот признак содержит дату отзывов, попробуем ее достать

### создадим новый признак - кол-во дней между отзывами

In [ ]:
# заполняем пропуски
data['Reviews'] = data['Reviews'].fillna("['no_Reviews']")
# создаем новую колонку и туда кладем только данные, ктр содержат дату
data['date_of_Review'] = data['Reviews'].str.findall('\d+/\d+/\d+')
# создаем новую колонку, ктр содержит разницу между значениями колонки date_of_Review и превращаем ее в дни
data['day_between_Reviews'] = data.apply(lambda x: pd.to_datetime(
    x['date_of_Review']).max() - pd.to_datetime(x['date_of_Review']).min(), axis=1).dt.days

In [ ]:
#проверяем, что получилось
data.info()

не все отзывы содержат дату, поэтому незаполненные значения заполним нулем

In [ ]:
data['day_between_Reviews'].fillna(0, inplace=True)

### Создадим новый признак - разница между отзывами больше/меньше год

добавление признака ухудшило МАЕ, закомментируем его

In [ ]:
#сравниваем сроки между отзывами с годом
#data['Old']=data['day_between_Reviews'].apply(lambda x: float(x)>float(365)) 
#конвертируем буллево значение в 0/1
#data['Old'] = data['Old'].astype(int)

### создадим новый признак - кол-во прошедших дней с последнего отзыва

In [ ]:
from datetime import datetime

In [ ]:
data['day_from_last_review'] = data.apply(lambda x: pd.datetime.now(
) - pd.to_datetime(x['date_of_Review']).max(), axis=1).dt.days

проверяем

In [ ]:
data.info()

есть пропуски, заполним их нулем

In [ ]:
data['day_from_last_review'].fillna(0, inplace=True)

__рассмотрим следующий признак Cuisine Style__

пропуски на other мы уже заменили, посмотрим, сколько всего разных кухонь, много ли уникальных значений

In [ ]:
# копируем датафрейм
data_copy = data.copy()
# создаем новую колонку в копии датафрейма - в каждой строке новой колонки список из рассплитованных  значений
data_copy['Cuisine'] = data['Cuisine Style'].str.findall(r"'(\b.*?\b)'")
# 'раздвигаем' исходный датасет, чтобы внутри признака было только одно значение вида кухни, а не список
data_copy = data_copy.explode('Cuisine')

In [ ]:
data_copy['Cuisine'].value_counts()

уникальных значений (кухонь, ктр встречаются только в одном ресторане) не так уж и много, всего 4. Поэтому не будем выносить это в отдельный признак

### создадим новый признак - кол-во кухонь в одном ресторане

In [ ]:
data['Number_of_cuisines'] = data['Cuisine Style'].apply(
    lambda x: len(x.split(',')))

посчитаем среднее значение для кол-ва кухонь в одном ресторане

In [ ]:
aver_cuis = data['Cuisine Style'].apply(
    lambda x: len(x.split(','))).sum()/len(data)

### создадим новый признак больше или меньше среднего значения кухонь в одном ресторане

добавление признакак ухудшило МАЕ, закомментируем его

In [ ]:
#сравниваем кол-во кухонь в ресторане со средним значением
#data['More/less_aver_cuis']=data['Number_of_cuisines'].apply(lambda x: x > aver_cuis) 
#конвертируем буллево значение в 0/1
#data['More/less_aver_cuis'] = data['More/less_aver_cuis'].astype(int)

# EDA 


### Посмотрим распределение признака Ranking

In [ ]:
plt.rcParams['figure.figsize'] = (10, 7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

Получаается, что признак Ranking несколько субъективен и зависит от количества ресторанов в городе, а много ресторанов именно в большом городе, потому что если ресторан занимает 10е место из 10000, то он отличный, а если 10е место из 20, то он может быть и средним, и вообще плохим, так как в в маленьком городе мало выбора.

Введем параметр Comp_Ranking - сравнительный ранг = ранг / кол-во ресторанов в городе.

### добавим новый признак  - сравнительный ранг ресторана в городе

In [ ]:
data['Comp_Ranking'] = data['Ranking'] / data['Rest_in_city']

### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### Корреляция

перед тем, как вывести тепловую карту с корреляцией, удалим все категориальные признаки

In [ ]:
data.drop(['Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA', 'date_of_Review'], axis=1, inplace=True)

In [ ]:
data.info()

In [ ]:
plt.rcParams['figure.figsize'] = (25,20)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Есть сильно скоррелированные параметры

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
#округлим предсказанные значения
y_pred = np.round(y_pred*2)/2

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission = np.round(predict_submission*2)/2

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)